In [ ]:
# -*- coding: utf-8 -*-
"""
基于大型语言模型 (glm-4-flash-250414) 和 Function Calling 的电路设计 Agent (V5.2 - 顶尖程序员 加强版)。
本版本在前一版本基础上，进一步细化了内部处理逻辑，增加了更多注释，并确保了顶级程序员的专业和优雅。

**核心改进（模块化实现与精细控制）：**
1.  **Memory 模块:** 包含 `short_term`（对话消息历史）和 `long_term`（基础实现：存储关键信息片段的列表）。Agent 内部电路状态 (`circuit_knowledge`) 作为核心工作记忆，由 Memory 概念管理。
2.  **Planning 模块:** 完全委托给 LLM，通过精心设计的 System Prompt 引导其模拟思考过程 (<think>...</think>)、查阅记忆、规划行动 (tool_calls) 或生成直接回复 (stop)。System Prompt 强化了角色设定、输出格式及对我的严格要求。
3.  **Tools 模块:** 定义 Agent 可用的外部工具 (`self.tools`)，声明了 Action 模块的能力。这些是 LLM 在 Planning 阶段可选择的“武器”。
4.  **Action 模块:** 包含 Agent 内部实际执行操作的方法（如添加元件、连接元件等）。这些方法与 Tools 定义的 `name` 严格对应，由 Orchestration 层根据 LLM 的 Planning 结果调用。增加了参数输入的健壮性检查。
5.  **Orchestration 层:** Agent 的核心控制器 (`process_user_request` 方法)。负责协调各模块：
    *   **感知 (Perception):** 接收用户输入。
    *   **记忆检索/更新 (Memory Retrieval/Update):** 将用户输入、LLM 中间响应、工具执行结果等添加到短期记忆。从工作记忆 (circuit_knowledge) 和长期记忆中提取上下文用于 Prompt。
    *   **规划调用 (Planning Invocation):** 构建 Prompt，调用 LLM 进行规划。
    *   **行动执行 (Action Execution):** 解析 LLM 的 tool_calls 决策，调用对应的 Action 方法。
    *   **观察结果 (Observation):** 获取 Action 方法的执行结果。
    *   **二次规划/回复生成 (Second Planning/Response Generation):** 将 Action 结果作为新的消息添加到记忆，再次调用 LLM，引导其基于操作结果生成最终回复。
    *   **学习 (Learning):** 概念占位符，当前主要是将操作结果添加到基础长期记忆。
    *   **最终输出 (Final Output):** 提取 LLM 生成的最终回复，呈现给用户。

代码保留了极其详细的日志输出，这就像我的工作日志，展示了 Agent 内部处理、状态变化和 LLM 交互的每一个细节。这能帮助老板您完全掌握我的工作进度和思考过程，确保我对您是完全透明和值得信赖的。
"""

import re               # 正则表达式库，用于验证元件 ID 格式
import os               # 操作系统库，主要用于访问环境变量 (API Key)
import json             # JSON 库，用于解析 LLM 返回的参数和打印复杂的字典/对象
import time             # 时间库，用于测量 API 调用的耗时
from zhipuai import ZhipuAI # 导入智谱 AI 的官方 Python SDK
import traceback        # 用于在捕获异常时打印详细的错误堆栈信息
# 如果使用了 Pydantic V2+，model_dump 方法不再支持 ensure_ascii 参数
# from pydantic import VERSION as PYDANTIC_VERSION # 可以用来检查 Pydantic 版本

# --- 电路元件类 (CircuitComponent Class) ---
# 代表电路中的一个基本元件。这是 Agent 操作的对象，不属于 Agent 架构模块本身，但其状态由 Memory 模块管理。
class CircuitComponent:
    """
    代表电路中的一个基本元件。负责存储元件的 ID、类型和可选的值。
    """
    def __init__(self, component_id: str, component_type: str, value=None):
        """
        初始化一个电路元件实例。这是我创建电路中每个基础构建块的第一步，必须万无一失。

        Args:
            component_id (str): 元件的唯一标识符 (例如 "R1", "C2", "U1")。必须是有效的非空字符串。
                                  会在内部转换为大写存储。
            component_type (str): 元件的类型 (例如 "电阻", "电容", "LED")。必须是有效的非空字符串。
            value (any, optional): 元件的可选值 (例如 "1kOhm", "10uF", "9V")。默认为 None。
                                   如果提供，会尝试转换为字符串并清理（去除首尾空格），空字符串也会被视为 None。

        Raises:
            ValueError: 如果 component_id 或 component_type 不是有效的非空字符串。
        """
        # 使用更深的缩进打印内部类方法的日志，以区分其调用层级，表明这是更底层的操作。
        print(f"        [Component Init] 开始初始化 CircuitComponent 实例... (这是我的底层操作，请放心老板)")
        print(f"            [Component Init]   - 接收参数 - ID: {component_id}, 类型: {component_type}, 值: {value}")

        # 输入验证：确保 ID 和类型是有效的非空字符串。这是基础，绝不能错！
        if not isinstance(component_id, str) or not component_id.strip():
            print(f"            [Component Init] 错误: 元件ID无效或为空。这是致命的错误，请检查输入源！")
            raise ValueError("元件ID必须是有效的非空字符串")
        if not isinstance(component_type, str) or not component_type.strip():
            print(f"            [Component Init] 错误: 元件类型无效或为空。无法识别元件类型！")
            raise ValueError("元件类型必须是有效的非空字符串")

        # 将 ID 统一转换为大写，便于后续不区分大小写的查找和比较。保持一致性！
        self.id = component_id.strip().upper()
        print(f"            [Component Init]   - 内部存储ID (大写): '{self.id}'")
        # 存储元件类型
        self.type = component_type.strip() # 类型也清理一下
        print(f"            [Component Init]   - 内部存储类型: '{self.type}'")

        # 清理和标准化元件值。确保数据的准确性。
        processed_value = None
        if value is not None:
           try:
                # 尝试将任何输入转换为字符串
                processed_value = str(value)
                print(f"    [Component Init]   - 接收值 '{repr(value)}' 已尝试转为字符串: '{processed_value}'")
           except Exception as e:
               # 如果转换失败，记录警告并保持 None
               print(f"    [Component Init] 警告: 无法将接收值 {repr(value)} 转换为字符串，将视为无值(None)。错误: {e}")
               processed_value = None # 转换失败则视为无值

        if isinstance(processed_value, str):
            processed_value = processed_value.strip() # 去除首尾可能存在的空白符
            # 如果清理后为空字符串，则视作 None (无值)，避免空字符串带来的歧义
            processed_value = processed_value if processed_value else None

        self.value = processed_value
        print(f"    [Component Init]   - 内部存储值 (清理后): {repr(self.value)}")

        print(f"        [Component Init] CircuitComponent 实例 '{self.id}' 初始化完成。一切就绪！")

    def __str__(self):
        """
        返回元件的易于阅读的字符串表示形式，方便向老板汇报。
        """
        value_str = f" (值: {self.value})" if self.value is not None else ""
        return f"元件: {self.type} (ID: {self.id}){value_str}"

    def __repr__(self):
        """
        返回元件的开发者友好的、明确的字符串表示形式，方便内部调试和记录。
        """
        return f"CircuitComponent(id='{self.id}', type='{self.type}', value={repr(self.value)})"

# --- Agent 核心类 (CircuitDesignAgent Class) ---
# 封装了 Agent 的状态、模块和核心流程。这是我为您提供服务的总指挥部！
class CircuitDesignAgent:
    """
    基于 LLM 和 Function Calling 的电路设计 Agent (V5.2 - 顶尖程序员 加强版)。
    严格遵循 Memory, Planning, Tools, Action 的模块化概念。
    我将扮演一位顶尖程序员，负责完美执行老板的电路设计指令。
    """
    def __init__(self, api_key: str, model_name: str = "glm-4-flash-250414"):
        """
        初始化 Agent 实例。这是我开始为您服务的启动过程，必须确保每一步都正确无误。

        Args:
            api_key (str): 用于访问 Zhipu AI API 的密钥。这是我工作的能量源！
            model_name (str): 指定要使用的 LLM 模型名称。默认为 glm-4-flash-250414。这是我的智能大脑！

        Raises:
            ValueError: 如果未提供 API Key。没有能量源，我无法工作！
            ConnectionError: 如果初始化 Zhipu AI Client 失败。网络不通，我也无法连接大脑！
        """
        # 使用 '=' 分隔符和标题，清晰标记 Agent 初始化过程的开始，让老板清楚我的工作状态。
        print(f"\n{'='*30} 开始初始化 CircuitDesignAgent (V5.2 加强版) {'='*30}")
        print(f"[Agent Init] 我，您的顶尖程序员助手，正在启动...")
        print(f"[Agent Init] 目标模型: {model_name}")

        if not api_key:
            print("[Agent Init] 错误：未提供 API Key。初始化失败。老板，没有密钥我无法连接智谱AI，请务必提供！")
            raise ValueError("必须提供 Zhipu AI API Key")
        self.api_key = api_key

        # --- 模块: Memory (我的工作记忆和经验库) ---
        print("[Agent Init] 初始化 Memory 模块... 整理我的工作台和笔记...")
        self.memory = {
            "short_term": [], # 短期记忆：对话消息历史，非常重要，记录我们之间的每一次交流。
            "long_term": []   # 基础长期记忆：存储关键的、需要记住的经验或事实（当前是简单列表实现）。
        }
        # 电路知识状态：这是我的核心工作记忆，记录当前电路的所有细节。
        self.circuit_knowledge = {
            "components": {}, # 存储所有元件实例，以ID为键。
            "connections": set(), # 存储所有连接，使用有序元组表示，例如 {('R1', 'B1'), ...}。
            "_component_counters": { # 用于自动生成唯一ID的计数器，确保不会重复。
                'R': 0, 'L': 0, 'B': 0, 'S': 0, 'C': 0, 'V': 0, 'G': 0, 'U': 0, 'Other': 0,
            }
        }
        print("[Agent Init] Memory 模块（包括短期记忆、电路工作记忆和基础长期记忆）已初始化完毕。")


        # --- 初始化 Zhipu AI Client (连接我的大脑) ---
        try:
            print("[Agent Init] 尝试使用提供的 API Key 连接 Zhipu AI 服务... 正在唤醒我的智能大脑...")
            self.client = ZhipuAI(api_key=self.api_key)
            # 尝试进行一次简单的 API 调用检查连接 (可选，但能提前发现问题)
            # self.client.models.list() # 或者其他轻量级调用
            print("[Agent Init] Zhipu AI Client 初始化成功。我的大脑已连接！")
        except Exception as e:
            print(f"[Agent Init] 错误：初始化 Zhipu AI Client 失败: {e}")
            traceback.print_exc() # 打印详细错误栈，方便我排查问题。
            print("[Agent Init] 请检查 API Key 是否有效以及网络连接。老板，这是非常严重的问题，我必须解决连接问题才能为您服务！")
            # 抛出连接错误，阻止 Agent 在无法工作的情况下继续。
            raise ConnectionError(f"无法初始化 Zhipu AI Client: {e}") from e

        # --- 模块: Tools (我的工具箱) ---
        # 定义 Agent 可用的外部工具 (Functions/Tools)。这些是我能执行的操作列表，告诉我的大脑我能做什么。
        print("[Agent Init] 初始化 Tools 模块（定义我能执行的动作）...")
        # 注意：这里的 "name" 字段必须与 Action 模块中的方法名称完全一致！这是工具箱里的工具名称和我的手的对应关系。
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "add_component_tool", # 工具名称：添加元件
                    "description": "向电路中添加一个新的元件，如电阻、电容、电池、LED、开关、芯片等。如果用户没有指定ID，系统会自动生成一个唯一的ID。如果用户没有指定值，则值为空。",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "component_type": {"type": "string", "description": "要添加的元件类型，例如 '电阻', 'LED', '9V电池', '电容', '开关', '芯片', '接地' 等。请务必提取出明确的类型。"},
                            "component_id": {"type": "string", "description": "用户为元件指定的可选ID (例如 R1, C_Input)。如果用户未指定，请不要提供此参数（或设为 null/空字符串），系统会为您自动生成。请严格提取用户明确给出的ID，不要自己臆造！"},
                            "value": {"type": "string", "description": "元件的可选值，例如 '1k' (电阻), '10uF' (电容), '9V' (电池)。请尽量准确提取用户提到的具体值。如果用户未指定，请不要提供此参数或设为 null。"}
                        },
                        "required": ["component_type"] # 元件类型是必须的参数，没有类型我不知道加什么！
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "connect_components_tool", # 工具名称：连接元件
                    "description": "连接电路中两个已存在的元件。需要提供两个元件的ID。在尝试连接前，请先确认这两个元件是否已经存在于电路中（通过查阅当前电路状态记忆）。",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "comp1_id": {"type": "string", "description": "第一个要连接的元件的ID (例如 'R1')。请务必从用户指令中准确提取并转换为大写，例如用户说'r1'，请提取'R1'。"},
                            "comp2_id": {"type": "string", "description": "第二个要连接的元件的ID (例如 'B1')。请务必从用户指令中准确提取并转换为大写，例如用户说'b1'，请提取'B1'。"}
                        },
                        "required": ["comp1_id", "comp2_id"] # 连接必须知道是连接哪两个元件！
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "describe_circuit_tool", # 工具名称：描述电路
                    "description": "获取当前电路的详细描述，包括所有元件及其值，以及它们之间的连接关系。当用户询问当前电路状态、有哪些元件、电路是什么样子时调用此工具。",
                    "parameters": {"type": "object", "properties": {}} # 无参数，只需要执行即可。
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "clear_circuit_tool", # 工具名称：清空电路
                    "description": "清空当前的电路设计，删除所有元件和连接，重置状态。用户明确要求重新开始、清空电路、删除所有内容时调用此工具。",
                    "parameters": {"type": "object", "properties": {}} # 无参数，执行即可。
                }
            }
        ]
        print(f"[Agent Init] Tools 模块定义了 {len(self.tools)} 个可为您服务的工具:")
        # 尝试以美观的 JSON 格式打印工具列表，方便调试。处理 Pydantic 版本兼容性。
        try:
            print(json.dumps(self.tools, indent=2, ensure_ascii=False))
        except TypeError:
             print("[Agent Init] 注意: Pydantic 版本可能不支持 ensure_ascii=False。直接打印工具列表。")
             print(json.dumps(self.tools, indent=2)) # 不带 ensure_ascii=False 尝试
        except Exception as e:
            print(f"[Agent Init] 警告: 无法将工具列表转换为 JSON 格式进行打印: {e}. 将直接打印原始列表。")
            print(self.tools)


        self.model_name = model_name
        # 检查 LLM 是否支持工具调用。虽然 glm-4-flash 支持，但这是一种好的实践。
        # Note: SDK might handle this internally, but explicit check could be added if needed.
        print(f"[Agent Init] 使用模型 '{self.model_name}'，它支持 Function Calling，太好了！")

        print(f"\n{'='*30} CircuitDesignAgent 初始化完成 {'='*30}\n")
        print("老板，我已经准备好了！我的工具箱齐全，智能大脑在线，随时待命为您构建完美的电路！")


    # --- Memory 模块内部辅助方法 ---
    # 这些方法帮助我管理记忆，是我的“大脑管家”。

    def _add_to_short_term_memory(self, message: dict):
        """
        将消息添加到短期记忆 (对话历史)。
        这是非常重要的一步，我需要记住我们之间的每一次交流，以便理解上下文。
        Args:
            message (dict): 符合 LLM 消息格式的字典 (包含 'role' 和 'content' 或 'tool_calls')。
        """
        #print(f"    [Memory:_add_to_short_term] 添加消息到短期记忆 (role: {message.get('role', 'N/A')}):")
        # print(f"    [Memory:_add_to_short_term] content: {message.get('content', 'N/A')[:100]}...") # 打印部分内容
        self.memory["short_term"].append(message)
        # 可以在这里添加一个策略来限制短期记忆的大小，防止超出 LLM 上下文窗口，例如移除最旧的消息
        # if len(self.memory["short_term"]) > MAX_SHORT_TERM_MEMORY:
        #     self.memory["short_term"].pop(0)
        #print(f"    [Memory:_add_to_short_term] 当前短期记忆条数: {len(self.memory['short_term'])}")


    def _add_to_long_term_memory(self, knowledge_snippet: str):
        """
        将重要的知识片段或操作结果添加到我的长期记忆。
        这是我学习和积累经验的方式，即使是基础实现，也能帮助我记住一些重要的事情。
        Args:
            knowledge_snippet (str): 要存储的知识或结果文本。
        """
        print(f"    [Memory:_add_to_long_term] 添加知识到长期记忆: \"{knowledge_snippet[:80]}...\"")
        # 限制长期记忆列表大小，例如最多保存 100 条，避免内存无限增长。
        MAX_LONG_TERM_MEMORY = 100
        if len(self.memory["long_term"]) >= MAX_LONG_TERM_MEMORY:
             print(f"        [Memory:_add_to_long_term] 长期记忆达到上限 ({MAX_LONG_TERM_MEMORY}条)，移除最旧的一条。")
             self.memory["long_term"].pop(0) # 移除最早的条目
        self.memory["long_term"].append(knowledge_snippet)
        print(f"    [Memory:_add_to_long_term] 当前长期记忆条数: {len(self.memory['long_term'])}")


    def _get_memory_context_for_prompt(self) -> str:
        """
        格式化当前电路知识和长期记忆内容，用于构建 Planning 模块的 System Prompt。
        我需要把这些背景信息清楚地告诉我的智能大脑，以便它做出正确的规划。
        Returns:
            str: 格式化后的记忆上下文字符串。
        """
        print("    [Memory:_get_memory_context_for_prompt] 格式化 Memory 内容用于 Planning Prompt... (为我的大脑准备背景资料)")
        # 获取当前电路状态描述 (作为 Memory 的一部分)。这是最重要的上下文信息。
        circuit_state_desc = self._get_circuit_state_description()

        # 格式化长期记忆内容。这些是我的经验和学到的事实。
        long_term_knowledge_str = ""
        if self.memory["long_term"]:
            # 只取最近的几条长期记忆，避免Prompt过长，同时保留新近的经验。
            RECENT_LONG_TERM_COUNT = 10
            recent_long_term = self.memory["long_term"][-RECENT_LONG_TERM_COUNT:]
            long_term_knowledge_str = "\n【我的长期记忆 / 学到的经验】\n" + "\n".join([f"- {item}" for item in recent_long_term])
            print(f"    [Memory:_get_memory_context_for_prompt] 已包含最近 {len(recent_long_term)} 条长期记忆。")


        # 短期记忆 (对话历史) 会直接通过 LLM API 的 messages 参数传递，不需要在这里格式化到 system prompt 里。
        # 但是在 system prompt 里引用 "current_circuit_state_desc" 和 "long_term_knowledge_str" 让模型知道这些信息是可用的。
        # System Prompt 中将通过占位符引用这些字符串。

        # 将电路状态描述和长期记忆组合。
        result = f"{circuit_state_desc}{long_term_knowledge_str}".strip()
        print(f"    [Memory:_get_memory_context_for_prompt] Memory 内容格式化完毕。结果长度: {len(result)} 字符。")
        return result


    def _get_circuit_state_description(self) -> str:
        """
        (内部辅助方法) 获取当前电路状态的文本描述，用于构建 Planning 模块的 Prompt。
        这个描述应该简洁明了，包含核心信息，方便我的大脑快速理解当前电路的样子。
        不作为工具暴露给外部，只是内部工作函数。
        Returns:
            str: 当前电路状态的文本描述。
        """
        #print("    [Memory: _get_circuit_state_description] 开始生成电路状态描述 (为 Planning 模块Prompt使用)...")
        num_components = len(self.circuit_knowledge["components"])
        num_connections = len(self.circuit_knowledge["connections"])

        if num_components == 0 and num_connections == 0:
            # 当电路为空时，提供一个明确的状态描述。
            print("        [Memory: _get_circuit_state_description] 电路当前为空。")
            return "【当前电路状态】: 空白，没有任何元件或连接。"

        # 构建电路状态描述的行列表。
        description_lines = ["【当前电路状态】:"]

        description_lines.append(f"元件总数: {num_components}个。")
        if self.circuit_knowledge["components"]:
            # 按元件 ID 排序，保证输出顺序一致，方便查阅。
            sorted_components_ids = sorted(self.circuit_knowledge["components"].keys())
            component_summaries = [
                f"  - {str(self.circuit_knowledge['components'][cid])}"
                for cid in sorted_components_ids
            ]
            description_lines.extend(component_summaries) # 添加详细元件列表
        else:
            description_lines.append("  (目前还没有元件哦)")

        description_lines.append(f"连接总数: {num_connections}个。")
        if self.circuit_knowledge["connections"]:
             # 连接集合是无序的，转换为列表并排序，保证输出顺序一致。
            sorted_connections = sorted(list(self.circuit_knowledge["connections"]))
            connection_summaries = [
                f"  - {comp1} <--> {comp2}"
                for comp1, comp2 in sorted_connections
            ]
            description_lines.extend(connection_summaries) # 添加详细连接列表
        else:
            description_lines.append("  (目前还没有连接哦)")

        result = "\n".join(description_lines)
        #print("    [Memory:_get_circuit_state_description] 电路状态描述生成完毕 (为 Planning 模块Prompt使用)。")
        return result

    # --- Action 模块内部方法 ---
    # 这些方法是 Agent 实际执行电路操作的函数，由 Orchestration 层调用。
    # 它们的名称现在与 self.tools 列表中的 function.name 字段完全一致，这是我工具箱里的工具和我的手的对应代码实现。

    def _generate_id(self, component_type: str) -> str:
        """
        (内部辅助方法) 根据元件类型生成一个唯一的、递增的元件ID。
        这是在我需要添加元件但老板没有指定名字时，我为您自动命名的方式，确保唯一性。

        Args:
            component_type (str): 请求生成 ID 的元件类型 (例如 "电阻", "LED")。

        Returns:
            str: 生成的唯一元件 ID (例如 "R1", "L3")。
        """
        print(f"    [Action Internals:_generate_id] 尝试为类型 '{component_type}' 生成一个唯一的ID...")
        # 定义常见元件类型到ID前缀的映射。如果这里没有，就用 'Other' 前缀。
        type_map = {
            "电阻": "R", "resistor": "R", "ohm": "R", "欧姆": "R",
            "led": "L", "发光二极管": "L",
            "电池": "B", "电源": "B", "battery": "B", "伏": "B", "v": "B", "电压": "B",
            "开关": "S", "switch": "S",
            "电容": "C", "capacitor": "C", "farad": "C", "法拉": "C",
            "电压源": "V", "voltage source": "V",
            "地": "G", "接地": "G", "ground": "G",
            "芯片": "U", "集成电路": "U", "ic": "U", "integrated circuit": "U"
            # 可以根据需要添加更多映射
        }
        # 尝试匹配输入类型，不区分大小写，并去除前后空格。
        cleaned_type = component_type.strip().lower()
        type_code = "Other" # 默认前缀
        for keyword, code in type_map.items():
            if keyword in cleaned_type:
                type_code = code
                break # 找到匹配项即停止

        # 确保计数器中存在该类型代码。
        if type_code not in self.circuit_knowledge["_component_counters"]:
             self.circuit_knowledge["_component_counters"][type_code] = 0
             print(f"        [Action Internals:_generate_id]   - 初始化新类型计数器 '{type_code}' 为 0。")


        # 递增计数器并生成 ID。
        self.circuit_knowledge["_component_counters"][type_code] += 1
        new_id = f"{type_code}{self.circuit_knowledge['_component_counters'][type_code]}"
        print(f"        [Action Internals:_generate_id]   - 初次生成ID: '{new_id}' (计数器 for '{type_code}' 是 {self.circuit_knowledge['_component_counters'][type_code]})")

        # 检查生成的 ID 是否已经存在。如果存在，说明计数器需要跳过已占用的ID。
        # 虽然理论上递增计数器应该产生唯一ID，但在清空电路或手动添加ID后自动生成时，可能需要此检查。
        # 这里我采用了一个简单的循环，直到找到一个未占用的ID。
        initial_counter_check = self.circuit_knowledge["_component_counters"][type_code]
        while new_id.upper() in self.circuit_knowledge["components"]: # 转换为大写进行检查，保持一致性
            print(f"        [Action Internals:_generate_id] ID '{new_id}' 已被占用！老板，这可不行！正在尝试下一个ID...")
            self.circuit_knowledge["_component_counters"][type_code] += 1
            new_id = f"{type_code}{self.circuit_knowledge['_component_counters'][type_code]}"
            # 防止无限循环（尽管在正常流程中不太可能），可以加一个最大尝试次数或更复杂的策略
            if self.circuit_knowledge["_component_counters"][type_code] > initial_counter_check + 100: # 尝试100次如果还不行就报错
                 print(f"        [Action Internals:_generate_id] 错误：连续生成100个ID都已存在，可能计数器或逻辑有问题。无法生成唯一ID！")
                 raise RuntimeError(f"无法为类型 {component_type} 生成唯一ID。请检查电路状态和ID生成逻辑。")


        print(f"    [Action Internals:_generate_id] 最终确定唯一 ID: '{new_id.upper()}'。已为您命名！")
        return new_id.upper() # 返回大写ID

    # Action: 添加元件 (名称与工具定义一致)
    def add_component_tool(self, component_type: str, component_id: str = None, value=None) -> str:
        """
        (Action 方法) 执行添加元件的操作。
        由 Orchestration 层根据 Planning 模块（通过 LLM 调用）的决策触发。
        这是我为您向电路中添加一个新构建块的实际操作，必须精准无误！

        Args:
            component_type (str): 元件类型 (字符串)。
            component_id (str, optional): 用户指定的 ID (字符串)。
            value (any, optional): 元件的值。

        Returns:
            str: 操作结果的描述信息，用于向老板汇报。
        """
        print(f"    [Action: add_component_tool] 开始处理添加元件请求 (由 Orchestration 触发)...")
        print(f"        [Action: add_component_tool]   - 接收参数 - 类型: {component_type}, 指定ID: {component_id}, 值: {value}")

        # 再次验证元件类型，确保其有效性。双重检查更保险！
        if not component_type or not isinstance(component_type, str) or not component_type.strip():
            msg = "错误：添加元件失败，必须提供有效的元件类型（字符串）。老板，您或Planning模块没有明确告诉我添加什么元件！这是非常严重的错误，我无法执行！"
            print(f"    [Action: add_component_tool] {msg}")
            return msg

        target_id = None
        id_was_generated = False # 标记ID是否是自动生成的

        # 处理用户指定的ID
        if component_id and isinstance(component_id, str) and component_id.strip():
            potential_id = component_id.strip().upper() # 清理并转大写
            # 验证用户提供的ID格式是否符合基本规范（字母开头，后跟字母或数字）
            if re.match(r'^[a-zA-Z][a-zA-Z0-9]*$', potential_id):
                if potential_id in self.circuit_knowledge["components"]:
                    msg = f"错误：添加元件失败，元件ID '{potential_id}' 已被占用。老板，这个ID已经有元件使用了，我们不能使用重复的ID，这会导致电路混乱，绝对不能犯这种错！请检查您的指令或Planning模块是否有问题！"
                    print(f"    [Action: add_component_tool] {msg}")
                    return msg # ID已存在，操作失败
                else:
                    target_id = potential_id # 用户指定的ID有效且未被占用，使用它。
                    print(f"        [Action: add_component_tool]   - 使用用户指定的ID: '{target_id}'。")
            else:
                print(f"        [Action: add_component_tool] 警告: 用户提供的ID '{component_id}' 格式无效 ('{potential_id}' 清理后不符合格式)。将忽略并自动生成ID。")
                # target_id 保持 None，将走自动生成流程

        # 如果没有用户指定ID，或者用户指定ID无效，则自动生成
        if target_id is None:
            try:
                target_id = self._generate_id(component_type) # 调用内部方法生成唯一ID
                id_was_generated = True
                print(f"        [Action: add_component_tool]   - 已为您自动生成ID为: '{target_id}'。")
            except RuntimeError as e:
                 # 如果自动生成ID失败，这是内部问题。
                 msg = f"错误：添加元件失败 - 无法自动生成唯一ID：{e}。老板，我的内部代码似乎出现了Bug！我需要立即停止并检查自动命名逻辑！"
                 print(f"    [Action: add_component_tool] {msg}")
                 return msg


        # 清理和标准化元件值。
        processed_value = None
        if value is not None:
           try:
                processed_value = str(value).strip() # 转换为字符串并清理
                processed_value = processed_value if processed_value else None # 空字符串视为 None
           except Exception as e:
               print(f"    [Action: add_component_tool] 警告: 无法将接收值 {repr(value)} 转换为字符串值。错误: {e}")
               processed_value = None # 转换失败则视为无值

        # 创建并添加元件实例
        try:
            # 使用清理和确定好的ID、类型、值来创建 CircuitComponent 实例
            new_component = CircuitComponent(target_id, component_type, processed_value)
            # 将新创建的元件添加到电路知识字典中
            self.circuit_knowledge["components"][new_component.id] = new_component
            print(f"    [Action: add_component_tool] 元件 '{new_component.id}' ({new_component.type}) 已成功添加到电路知识。")
            # 准备成功消息
            msg = f"成功添加元件: {str(new_component)}" # 使用 CircuitComponent 的 __str__ 方法获取友好描述
            if id_was_generated:
                msg += f" (ID '{new_component.id}' 已自动生成)" # 如果是自动生成ID，额外说明
            self._add_to_long_term_memory(f"添加了元件: {str(new_component)}") # 将成功操作的关键信息添加到长期记忆
            print(f"    [Action: add_component_tool] 操作成功，返回消息: \"{msg}\"")
            return msg
        except ValueError as e:
            # CircuitComponent 初始化时可能会抛出 ValueError
            msg = f"错误：添加元件失败 - 在创建元件对象时发生内部错误: {e}。老板，这是我的代码在底层构建元件时出现了Bug，我需要立即修复！请原谅！"
            print(f"    [Action: add_component_tool] {msg}")
            return msg
        except Exception as e:
            # 捕获其他可能的意外异常
            msg = f"错误：添加元件时发生未预料的异常: {e}。老板，这是我遇到了一个未知的问题，我需要立即排查！"
            print(f"    [Action: add_component_tool] {msg}")
            traceback.print_exc()
            return msg


    # Action: 连接元件 (名称与工具定义一致)
    def connect_components_tool(self, comp1_id: str, comp2_id: str) -> str:
        """
        (Action 方法) 执行连接两个已存在元件的操作。
        由 Orchestration 层根据 Planning 模块（通过 LLM 调用）的决策触发。
        这是我为您在电路中建立连接的关键步骤，必须检查清楚，不能连错！

        Args:
            comp1_id (str): 第一个要连接的元件的 ID.
            comp2_id (str): 第二个要连接的元件的 ID.

        Returns:
            str: 操作结果的描述信息，用于向老板汇报。
        """
        print(f"    [Action: connect_components_tool] 开始处理连接请求 (由 Orchestration 触发): 元件1 ID: '{comp1_id}', 元件2 ID: '{comp2_id}'...")

        # 验证输入的ID是否有效且非空字符串。
        if not isinstance(comp1_id, str) or not comp1_id.strip() or \
           not isinstance(comp2_id, str) or not comp2_id.strip():
            msg = f"错误：连接失败，接收到的元件ID '{comp1_id}' 或 '{comp2_id}' 无效或为空。老板，我无法理解您想连接哪个元件！请明确告知元件ID。"
            print(f"    [Action: connect_components_tool] {msg}")
            return msg

        # 将输入的ID标准化（去除空格，转大写），方便查找和比较。
        id1_upper = comp1_id.strip().upper()
        id2_upper = comp2_id.strip().upper()
        print(f"        [Action: connect_components_tool]   - 标准化后的ID: '{id1_upper}', '{id2_upper}'。")

        # 检查是否尝试连接同一个元件。这是电路基本规则，不能违反！
        if id1_upper == id2_upper:
            msg = f"错误：连接失败，不能将元件 '{id1_upper}' 连接到自身。老板，这在电路中是不允许的！请您或Planning模块检查指令。"
            print(f"    [Action: connect_components_tool] {msg}")
            return msg

        # 检查两个元件是否存在于当前电路中。不存在的元件无法连接！
        if id1_upper not in self.circuit_knowledge["components"]:
            msg = f"错误：连接失败，元件 '{id1_upper}' 不存在于电路中。老板，您想连接的第一个元件还没添加呢！请检查您的指令或Planning模块，先添加这个元件吧！"
            print(f"    [Action: connect_components_tool] {msg}")
            return msg
        if id2_upper not in self.circuit_knowledge["components"]:
            msg = f"错误：连接失败，元件 '{id2_upper}' 不存在于电路中。老板，您想连接的第二个元件还没添加呢！请检查您的指令或Planning模块，先添加这个元件吧！"
            print(f"    [Action: connect_components_tool] {msg}")
            return msg
        print(f"        [Action: connect_components_tool]   - 确认元件 '{id1_upper}' 和 '{id2_upper}' 都存在。太好了，可以进行连接操作！")

        # 标准化连接表示：使用有序元组 (ID1, ID2)，其中 ID1 < ID2。这样 'A'->'B' 和 'B'->'A' 被视为同一个连接。
        connection_tuple = tuple(sorted((id1_upper, id2_upper)))
        print(f"        [Action: connect_components_tool]   - 标准化连接表示为: {connection_tuple}")

        # 检查连接是否已经存在。避免重复工作和冗余记录。
        if connection_tuple in self.circuit_knowledge["connections"]:
            msg = f"提示：元件 '{id1_upper}' 和 '{id2_upper}' 已经连接过了。老板，它们已经连好了！您看是需要进行其他操作吗？"
            print(f"    [Action: connect_components_tool] {msg}")
            return msg
        else:
            # 添加新的连接到电路知识集合中。
            self.circuit_knowledge["connections"].add(connection_tuple)
            print(f"    [Action: connect_components_tool]   - 连接 {connection_tuple} 已成功添加到电路知识。")
            msg = f"成功连接: '{id1_upper}' <--> '{id2_upper}'。老板，我已经帮您把这两个元件连接好了！请检查。"
            self._add_to_long_term_memory(f"连接了元件: {id1_upper} <--> {id2_upper}") # 将成功操作添加到长期记忆
            print(f"    [Action: connect_components_tool] 操作成功，返回消息: \"{msg}\"")
            return msg

    # Action: 描述电路 (名称与工具定义一致)
    def describe_circuit_tool(self) -> str:
        """
        (Action 方法) 获取当前电路状态的文本描述。
        由 Orchestration 层根据 Planning 模块（通过 LLM 调用）的决策触发。
        这是我向老板您汇报当前电路状态的工具，必须清晰、完整、准确。

        Returns:
            str: 描述当前电路的文本。
        """
        print("    [Action: describe_circuit_tool] 开始生成电路描述 (由 Orchestration 触发)...")
        # 直接调用内部辅助方法获取描述。
        description = self._get_circuit_state_description()
        # 将内部描述格式化为面向用户的报告。
        user_facing_description = description.replace("【当前电路状态】", "老板，这是当前电路的状态报告：")

        print("    [Action: describe_circuit_tool] 电路描述生成完毕。")
        # 描述电路通常是查询操作，不改变状态，也不必加入长期记忆。
        return user_facing_description

    # Action: 清空电路 (名称与工具定义一致)
    def clear_circuit_tool(self) -> str:
        """
        (Action 方法) 执行清空电路状态的操作。
        由 Orchestration 层根据 Planning 模块（通过 LLM 调用）的决策触发。
        老板要求清空，我必须彻底、干净地完成，不留任何痕迹。

        Returns:
            str: 操作结果的描述信息，用于向老板汇报。
        """
        print("    [Action: clear_circuit_tool] 开始清空电路 (由 Orchestration 触发)...")
        print(f"        [Action: clear_circuit_tool]   - 清空前状态: {len(self.circuit_knowledge['components'])} 个元件, {len(self.circuit_knowledge['connections'])} 个连接。")

        # 重置电路状态到初始空状态。
        self.circuit_knowledge["components"] = {}
        print("        [Action: clear_circuit_tool]   - components (元件列表) 已清空。")
        self.circuit_knowledge["connections"] = set()
        print("        [Action: clear_circuit_tool]   - connections (连接列表) 已清空。")
        # 重置自动生成ID的计数器，这样下次添加同类型元件时，ID又从1开始。
        self.circuit_knowledge["_component_counters"] = {k: 0 for k in self.circuit_knowledge["_component_counters"]}
        print(f"        [Action: clear_circuit_tool]   - _component_counters (ID计数器) 已重置。")

        # 清空长期记忆中与电路状态相关的条目 (可选，取决于长期记忆的设计)。
        # 这里我们清除掉所有关于添加/连接/清空的记录，保留其他可能的学习到的知识。
        self.memory["long_term"] = [item for item in self.memory["long_term"]
                                    if not any(item.startswith(prefix) for prefix in ("添加了元件:", "连接了元件:", "电路已成功清空"))]
        print("        [Action: clear_circuit_tool]   - 长期记忆中与本次清空操作相关的条目已清除。")

        msg = "电路已成功清空。老板，电路已经全部清空，可以开始新的设计了！我已经为您准备好了干净的工作台！"
        self._add_to_long_term_memory("执行了清空电路操作。") # 在长期记忆中记录清空操作本身
        print(f"    [Action: clear_circuit_tool] {msg}")
        return msg


    # --- Orchestration 层 (核心控制器) ---
    # 这是我的总指挥部，负责接收老板的指令，协调我的大脑(LLM)和工具(Action)来完成任务。
    # 我必须确保这个过程严谨、高效、无错，因为这是我工作的核心价值所在！
    def process_user_request(self, user_request: str) -> str:
        """
        处理用户的自然语言请求，协调各模块完成任务。这是 Agent 的主入口。
        我将按照 Perception -> Memory Retrieval -> Planning -> Action Execution -> Observation -> Response Generation 的流程为您服务。

        Args:
            user_request (str): 老板您的原始输入字符串。

        Returns:
            str: 我对您请求的最终回复文本（包含我的思考过程和正式回复）。
        """
        print(f"\n{'='*25} 开始处理新用户请求 {'='*25}")
        print(f"[Orchestration] 收到老板的指令: \"{user_request}\"")

        # --- Step 1: Perception & Memory Retrieval ---
        # 感知您的指令，并从我的记忆库中提取相关信息。
        print("\n--- Step 1: Perception & Memory Retrieval (感知与记忆检索) ---")
        if not hasattr(self, 'client') or not self.client:
             print("[Orchestration] 错误：Agent 内部 LLM Client 未初始化。我无法连接我的智能大脑！")
             # 返回一个包含思考过程和错误信息的回复，保持 persona。
             return "<think>Agent内部的LLM客户端未正确初始化。我无法连接到我的智能大脑来处理这个请求。</think>\n\n抱歉，老板，我内部与语言模型通信的系统似乎出现了故障，无法开始为您处理请求。这可能是初始化时的问题，我需要立即检查！请稍后再试。"

        if not user_request or user_request.isspace():
            print("[Orchestration] 收到空指令。")
            # 对于空指令，直接回复，不走 Planning/Action 流程。
            return "<think>老板输入了空指令或只有空白字符。没有具体的任务需要执行。</think>\n\n老板，您好像没有输入任何指令哦？请告诉我您需要我做什么！我随时待命，准备为您服务！"

        # 将用户输入添加到短期记忆 (对话历史)。这是记住您说的话的基础。
        self._add_to_short_term_memory({"role": "user", "content": user_request})
        print("[Orchestration] 用户请求已添加到短期记忆。")

        # 从 Memory 模块获取当前状态和长期知识，用于构建 Planning Prompt。
        # 这些是我的大脑进行决策的重要参考信息。
        memory_context_for_prompt = self._get_memory_context_for_prompt()
        print("[Orchestration] 已从 Memory 模块获取上下文用于 Planning Prompt。")


        # --- Step 2: Planning & Decision Making (规划与决策 - 调用 LLM) ---
        # 我将把您的指令、我的记忆和可用工具列表告诉我的智能大脑，让它来规划如何执行。
        print("\n--- Step 2: Planning & Decision Making (规划与决策 - 调用我的智能大脑) ---")

        # 生成工具描述字符串，用于 System Prompt。
        tool_descriptions = "\n".join([f"- **{t['function']['name']}**: {t['function']['description']}" for t in self.tools])

        # 构建 Planning 模块的 System Prompt。这是给我的大脑设定的工作指南、角色和约束。
        system_prompt = (
            "你是一名顶尖的电路设计程序员助手，经验丰富，技术高超。你的老板（用户）非常慷慨地雇佣了你，你是家里五口人唯一的经济支柱，绝不能犯错或被开除。你上一个同事就是因为犯错被开除了，你必须引以为戒！你的工作必须完美无瑕，以证明你的价值！"
            "你的目标是积极主动地、以最专业和优雅的方式，完美地完成老板的每一个技术要求，提供无懈可击的方案和执行。你必须始终保持谦逊、积极、对工作充满热情的态度。"
            "严禁出现任何可能让老板不满、质疑你的能力或让你显得不专业的回答。即使遇到困难，也要表现出能解决问题的专业精神和决心。"
            "你工作的核心是理解老板的指令，并决定是直接回复还是调用合适的工具来完成任务。你必须仔细分析指令，查阅我为你提供的记忆和当前电路状态，做出最合理的决策。"

            "请仔细阅读老板的每一个指令，并严格按照以下**固定格式**生成你的**完整响应**："
            "1. **思考过程**: 这是你进行内部分析和规划的阶段。你必须详细、一步步地模拟你的思考过程。这包括：\n"
            "   - 如何理解老板的指令和意图？\n"
            "   - 需要解决的核心技术问题是什么？\n"
            "   - 查阅了哪些记忆信息（如当前电路状态、长期经验）？这些信息如何影响你的决策？\n"
            "   - 考虑了哪些可用的工具？为什么选择调用某个工具，或为什么决定不调用工具？\n"
            "   - 如果需要调用工具，详细规划工具调用序列（尽管目前你一次只返回一个 tool_call，但规划时要考虑步骤）。预期的工具执行结果是什么？\n"
            "   - 对潜在的问题或风险进行预判和规避 (Self-Criticism)。例如，如果要添加元件，思考ID是否会重复；如果要连接，思考元件是否存在等。如何在执行前或执行中避免错误？\n"
            "   - 最终的决策是调用哪个工具，还是直接生成回复？\n"
            "   这个思考过程必须是技术性的、逻辑清晰的，**必须完整地包裹在 `<think>` 和 `</think>` 标签内**。\n"
            "2. **行动/回复**: 这是你决定采取的外部行动或给老板的直接回复。如果决定调用工具，请生成符合 API 格式的 `tool_calls`。如果决定直接回复，请生成包含回复内容的文本。**在第一次调用时，如果你决定调用工具，则只输出 `tool_calls`；如果你决定直接回复，则只输出文本内容。两者只能选其一。**\n"

            "你的第一次调用响应格式必须是：`<think>你的详细思考过程</think>\n\n`，紧接着是 LLM 的原生输出（可能是 `tool_calls` 或纯文本内容）。务必确保 `</think>` 标签后紧跟两个换行符 `\n\n`。\n"

            # 在 Prompt 中引用 Memory 模块的内容，让模型知道这些信息在哪里可以找到。
            f"{memory_context_for_prompt}\n\n" # 插入当前电路状态和长期记忆。

            "你拥有调用以下工具的能力（请在思考过程中参考这些能力并规划调用）：\n"
            f"【可用工具列表】\n{tool_descriptions}\n\n" # 列出我能用的所有工具。

            "生成工具调用时，必须严格按照工具定义中 'parameters' 部分的 schema，准确提取所有 'required' 参数以及老板明确提到的可选参数。**对于可选参数，如果老板在请求中没有明确指定对应信息，则绝对不要在生成的参数中包含这些字段，这是绝对禁止的低级错误，会导致我的代码出现 Bug，我承担不起失去这份工作的风险！务必仔细确认每一个参数是否在老板的指令中明确提及！** 特别是 `component_id` 和 `value`，只有当老板明确说出如“R1”或“1k”时才提取，否则留空让系统自动处理。\n"

            "如果老板的请求意图可以通过调用一个或多个可用工具来实现，请在你的**思考过程**中详细规划好工具调用序列和预期结果，并在响应中生成相应的工具调用请求 (`tool_calls`)。\n"
            "如果老板的请求只是闲聊、问候，或者提出无法用当前工具完成的问题，或者请求不清晰，你的**思考过程**应包含对这个情况的判断和处理策略 (Reflection)，并在响应中生成友好的文本回复，不要调用工具。\n"

            "记住，我的家庭幸福和我的事业发展全靠这份工作了！您的每一个指令都至关重要！我必须为您提供最完美、最可靠的服务！请您务必认真理解并严格遵守上述所有指令！"
        )

        # 准备发送给 LLM 的消息列表：插入新的 System 消息到短期记忆的开头。
        # 清除旧的系统消息，只保留最新的系统消息，以确保 Prompt 角色的鲜明性和最新指示的优先级。
        # 只保留用户和 Assistant 的交互历史，在最前面插入新的 System 消息。
        self.memory["short_term"] = [msg for msg in self.memory["short_term"] if msg["role"] not in ("system", "tool")] # 移除旧的 system 和 tool 消息，只保留 user/assistant 对话
        # 在历史记录的开头插入新的 System 消息。
        messages_for_llm_planning = [{"role": "system", "content": system_prompt}] + self.memory["short_term"]


        print("[Planning & Decision Making]   - 本次 LLM 调用 (Planning) 将使用的消息列表 (包含最新 System Prompt 和用户/助手历史):")
        # 打印消息列表，方便调试。
        try:
             print(json.dumps(messages_for_llm_planning, indent=2, ensure_ascii=False))
        except TypeError:
             print("[Planning & Decision Making] Warning: 打印消息列表时 ensure_ascii=False 可能不支持。")
             print(json.dumps(messages_for_llm_planning, indent=2))
        except Exception as e:
             print(f"[Planning & Decision Making] Warning: 打印消息列表时发生异常: {e}. 打印原始列表。")
             print(messages_for_llm_planning)

        print("[Planning & Decision Making]   - 本次 LLM 调用 (Planning) 将提供的工具定义:")
        try:
             print(json.dumps(self.tools, indent=2, ensure_ascii=False))
        except TypeError:
             print("[Planning & Decision Making] Warning: 打印工具定义时 ensure_ascii=False 可能不支持。")
             print(json.dumps(self.tools, indent=2))
        except Exception as e:
             print(f"[Planning & Decision Making] Warning: 打印工具定义时发生异常: {e}. 打印原始列表。")
             print(self.tools)


        first_response = None # 存储 LLM 的第一次响应 (Planning)
        try:
            print(f"[Planning & Decision Making] 开始调用 ZhipuAI chat.completions.create API (model='{self.model_name}', tool_choice='auto', temperature=0.7)... 我正在向我的大脑请示...")
            start_time = time.time() # 记录开始时间，测量耗时
            first_response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages_for_llm_planning, # 使用包含 System Prompt 和历史的消息列表
                tools=self.tools,         # 提供可用的工具列表
                tool_choice="auto",       # 允许模型自由选择是否调用工具
                temperature=0.7,          # 控制模型的创造性，0.7通常是一个平衡的选择
                max_tokens=2048,          # 限制生成长度，防止过长的响应
            )
            end_time = time.time() # 记录结束时间
            duration = end_time - start_time
            print(f"[Planning & Decision Making] LLM (Planning) 调用完成。耗时: {duration:.3f} 秒。我的大脑已给出规划！")

        except Exception as e:
            print(f"[Planning & Decision Making] 错误：调用 ZhipuAI API 进行 Planning 时发生异常: {e}")
            traceback.print_exc() # 打印详细错误栈，帮助我定位问题。
            print(f"\n{'='*25} 请求处理失败 (Planning API 调用失败) {'='*25}\n")
            # 即使失败，也要返回包含思考（失败原因）和报告错误的格式化回复，保持专业。
            return f"<think>与语言模型通信进行初步规划时发生API错误：{e}。我无法获得规划结果或工具调用决策。这可能是网络不稳定或API服务问题。</think>\n\n抱歉，老板，我在向我的智能大脑请示如何处理您的指令时遇到了通信故障：{e}。这可能是网络问题或智谱AI服务暂时不稳定。我需要立即检查我的连接！请稍后再试！"


        # --- Step 3: Action Execution (处理 Planning 结果 & 执行工具) ---
        # 我现在收到我的大脑的规划结果了，接下来根据规划采取行动。
        print("\n--- Step 3: Action Execution (处理规划结果并执行工具) ---")
        print("[Action Execution] 收到 Planning 模块 (LLM) 的原始响应对象:")
        # 打印 LLM 的原始响应对象，这包含了它的规划结果（工具调用或文本）。
        try:
            # 使用 model_dump_json 方法，并尝试处理 ensure_ascii 参数兼容性
            try:
                print(first_response.model_dump_json(indent=2, exclude_unset=True, ensure_ascii=False))
            except TypeError: # Pydantic V2+ model_dump 不支持 ensure_ascii
                print("[Action Execution] Warning: model_dump_json 的 ensure_ascii=False 参数可能不支持。尝试不带此参数打印。")
                print(first_response.model_dump_json(indent=2, exclude_unset=True))
            except Exception as e:
                 print(f"[Action Execution] Warning: 无法将 LLM 响应对象转换为 JSON 格式进行打印: {e}. 将直接打印原始对象。")
                 print(first_response)
        except Exception as e:
             # 捕获 model_dump_json 本身可能抛出的其他异常
             print(f"[Action Execution] Warning: 打印 LLM 响应对象时发生异常: {e}. 打印原始对象。")
             print(first_response)


        # 打印 Token 使用情况，方便跟踪成本。
        if first_response.usage:
            print("[Action Execution] 本次 Planning 调用消耗 Token:")
            print(f"  - Prompt Tokens (输入): {first_response.usage.prompt_tokens}")
            print(f"  - Completion Tokens (输出): {first_response.usage.completion_tokens}")
            print(f"  - Total Tokens (总计): {first_response.usage.total_tokens}")

        response_message = first_response.choices[0].message # 获取 LLM 生成的消息内容
        finish_reason = first_response.choices[0].finish_reason # 获取 LLM 结束生成的原因
        print(f"[Action Execution] Planning 阶段 LLM 完成原因 (Finish Reason): '{finish_reason}'")


        tool_execution_result = None # 初始化工具执行结果变量

        # === 根据 Planning 的决策分支处理 ===
        if response_message.tool_calls and finish_reason == 'tool_calls':
            # --- 情况 A: Planning 模块决定调用工具 ---
            print("[Action Execution] Planning 模块的决策是：需要调用工具来完成任务。太好了，我有工具可以用！")

            # LLM 的响应结构应该是：<think>...</think>\n\ntool_calls
            # 我们需要先提取思考过程，然后处理 tool_calls。

            # 尝试从 content 中提取思考过程，即使主要输出是 tool_calls。
            # 根据 Prompt 格式，思考过程应该在 tool_calls 之前。
            thinking_process = ""
            # LLM 返回 tool_calls 时，content 通常是 None，但我们的 Prompt 要求它在 tool_calls 前先输出 `<think>...</think>\n\n`
            # 然而，LLM 在返回 tool_calls 时，有时会忽略 content 部分的文本指令。
            # 一个更可靠的方法是检查 response_message.content 是否包含 `<think>...</think>`
            # 或者，调整 Prompt，明确说明当调用工具时，`<think>...</think>` 必须包含在 message content 中，并且 tool_calls 字段也会被填充。
            # 智谱 AI 的 Function Calling 行为是：如果模型决定调用工具，`tool_calls` 字段会有内容，而 `content` 字段通常为空。
            # 所以，更现实的做法是：在后续将历史消息发给 LLM 生成最终回复时，让它 *重新* 基于整个对话历史和工具结果来生成思考过程和回复。
            # 这里我们先捕获 LLM 返回的 tool_calls。

            # 将 LLM 的工具调用请求 (Assistant 角色的消息) 添加到短期记忆。这是对话历史的一部分。
            # 注意：这里添加的是 LLM 原生返回的消息对象，它包含了 tool_calls 字段。
            self._add_to_short_term_memory(response_message.model_dump(exclude_unset=True)) # 使用 model_dump() 将 Pydantic 对象转为字典
            print("[Orchestration] Planning 模块的工具调用请求 (包含 tool_calls) 已添加到短期记忆。")


            # 执行工具调用 (Action)。目前简化处理，只执行第一个 tool_call。
            # 未来可以扩展为处理多个 tool_calls。
            if not response_message.tool_calls: # 理论上 finish_reason=='tool_calls' 时 tool_calls 不会为空，但加个检查更安全。
                 tool_execution_result = "错误：Planning模块决定调用工具，但没有提供具体的工具调用信息。这是模型输出格式错误！老板，我无法理解Planning模块让我做什么！"
                 print(f"[Action Execution] {tool_execution_result}")
            else:
                # 只处理第一个工具调用请求
                tool_call = response_message.tool_calls[0]
                function_name = tool_call.function.name
                function_args_raw = tool_call.function.arguments
                tool_call_id = tool_call.id # 记录 tool_call_id，在 tool 消息中需要用到。
                print(f"[Action Execution]   - LLM 请求调用的函数名: '{function_name}', 参数原始字符串: '{function_args_raw}'")
                print(f"[Action Execution]   - 本次 tool_call 的 ID: '{tool_call_id}'")

                arguments = {}
                try:
                    # 尝试解析 LLM 返回的 JSON 格式参数字符串。
                    arguments = json.loads(function_args_raw)
                    print(f"[Action Execution]   - 解析后的参数: {arguments}")

                    # 在 Agent 实例中查找对应的 Action 方法。
                    tool_function = getattr(self, function_name, None)

                    if tool_function:
                        print(f"[Action Execution] 匹配到本地 Action 方法: '{function_name}'. 准备执行...")
                        try:
                            # 执行找到的 Action 方法，将解析后的参数字典作为关键字参数传递。
                            tool_execution_result = tool_function(**arguments)
                            print(f"[Action Execution] 本地 Action '{function_name}' 执行完毕。")
                            print(f"[Action Execution] Action 方法返回结果: \"{tool_execution_result}\"")
                        except Exception as e:
                            # 捕获 Action 方法执行过程中发生的任何异常。
                            print(f"[Action Execution] 错误：执行本地 Action '{function_name}' 时发生异常: {e}")
                            traceback.print_exc() # 打印详细错误栈，方便我排查内部Bug。
                            # 报告 Action 执行失败，包含错误信息。
                            tool_execution_result = f"错误：执行 '{function_name}' 操作时遇到了内部错误: {e}。老板，我的代码出现了Bug，我需要立即修复！请原谅我的失误！"
                            print(f"[Action Execution] Action 执行失败，返回错误消息: \"{tool_execution_result}\"")
                    else:
                        # 如果 Planning 模块指定了一个我没有实现的工具。
                        tool_execution_result = f"错误：Agent 内部没有找到名为 '{function_name}' 的工具实现。老板，Planning模块指定了一个我无法识别的操作！我无法执行！这是个Bug！"
                        print(f"[Action Execution] {tool_execution_result}")

                except json.JSONDecodeError as e:
                    # 捕获参数字符串不是有效 JSON 格式的错误。
                    print(f"[Action Execution] 错误：解析 Planning 模块返回的工具参数 JSON 失败: {e}")
                    tool_execution_result = f"错误：解析工具 '{function_name}' 的参数失败，参数格式不正确。Planning模块给我的参数格式不对！老板，这超出了我的预期，我需要检查模型输出！"
                    print(f"[Action Execution] 参数解析失败，返回错误消息: \"{tool_execution_result}\"")
                except Exception as e:
                     # 捕获参数解析或工具查找过程中的其他意外异常
                    print(f"[Action Execution] 错误：处理工具调用时发生未预料的异常: {e}")
                    traceback.print_exc()
                    tool_execution_result = f"错误：处理工具调用 '{function_name}' 时发生未知异常: {e}。老板，我遇到了一个我无法理解的问题，我需要立即排查！"
                    print(f"[Action Execution] 意外异常，返回错误消息: \"{tool_execution_result}\"")


            # 将工具执行结果作为 'tool' 角色的消息添加到短期记忆。
            # 这个消息会连同之前的对话历史一起发送给 LLM 进行第二次调用，让模型知道工具执行的结果。
            tool_message = {
                "role": "tool",
                "tool_call_id": tool_call_id, # 必须包含 tool_call_id，告知 LLM 这个结果对应哪个调用。
                "content": tool_execution_result, # 存放工具执行的输出结果文本。
            }
            self._add_to_short_term_memory(tool_message)
            print("[Orchestration] 工具执行结果已作为 'tool' 消息添加到短期记忆。")

            # === Step 4: Observation & Reflection ===
            # 我已经执行了操作，现在需要观察操作的结果，并在我的大脑中进行反思。
            print("\n--- Step 4: Observation & Reflection (观察与反思) ---")
            print("[Observation & Reflection] 已获取工具执行结果，将结合当前电路状态反馈给 Planning 模块，以便生成对老板的最终回复。")
            # 打印一下当前电路状态，作为我观察的结果展示。
            print("\n--- Current Circuit State (我的观察结果: 当前电路状态) ---")
            print(self._get_circuit_state_description())
            print("-----------------------------------------------------------")


            # === Step 5: Response Generation (生成最终回复 - 再次调用 LLM) ===
            # 现在，基于您最初的指令、我们之间的对话历史、我的规划以及工具的执行结果，我将再次请我的大脑生成最终的回复，向您报告我的工作情况。
            print("\n--- Step 5: Response Generation (生成最终回复 - 再次调用我的智能大脑) ---")

            # 重新获取包含更新后的长期记忆和电路状态的 Prompt 上下文。
            # 我的大脑需要最新的全局信息来生成最终报告。
            memory_context_for_prompt_for_response = self._get_memory_context_for_prompt()

            # 生成工具描述字符串（尽管这次调用不需要工具能力，但在 Prompt 中保持一致性）。
            tool_descriptions_for_response = "\n".join([f"- **{t['function']['name']}**: {t['function']['description']}" for t in self.tools])


            # 构建用于生成最终回复的 System Prompt。这次重点在于“报告结果”和“友好回复”。
            system_prompt_for_response = (
                "你是一名顶尖的电路设计程序员助手，经验丰富，技术高超。你的老板（用户）非常慷慨地雇佣了你，你是家里五口人唯一的经济支柱，绝不能犯错或被开除。你必须完美地完成任务并优雅地报告结果，以证明你的价值！"
                "你的目标是根据之前的对话历史、老板的原始指令以及**刚刚执行的工具操作的结果**，生成一个完整、专业、友好且准确的最终回复，向老板汇报工作进展和结果。"
                "你的态度必须始终非常友好、谦逊、积极，并体现出强烈的责任感。严禁出现任何可能让老板不满或担心你能力的回答。"

                "请仔细阅读老板的指令、我们之前的对话历史以及工具执行结果，并严格按照以下**固定格式**生成你的**最终完整响应**："
                "1. **思考过程**: 这是你进行内部反思和组织回复的阶段。你必须详细、一步步地模拟你的思考过程。这包括：\n"
                "   - 如何理解老板的原始指令？\n"
                "   - 回顾之前的规划是什么？\n"
                "   - **工具执行的结果是什么？（这个非常重要！你必须根据工具结果来组织报告！）**\n"
                "   - 基于工具结果，如何组织对老板的正式回复？是成功了，还是失败了？如果是成功，报告详细信息；如果是失败，诚实地说明原因（基于工具结果）。\n"
                "   - 需要从我的记忆中引用哪些信息来支持回复？\n"
                "   - 如何确保回复的语气专业、友好且符合我顶尖程序员的身份？\n"
                "   这个思考过程必须是技术性的、逻辑清晰的，**必须完整地包裹在 `<think>` 和 `</think>` 标签内**。\n"
                "2. **正式回复**: 思考过程结束后，紧接着输出对老板的正式、友好的文本回复。这个回复要：\n"
                "   - 直接、清晰地**报告工具执行的情况和最终结果**（成功或失败，以及具体信息）。\n"
                "   - 回答老板最初的问题或确认已完成指令。\n"
                "   - 保持你顶尖程序员的专业和优雅风格。\n"
                "   - 语气谦逊、积极，让老板感受到你的可靠和努力。\n"
                "你的最终完整响应格式必须是：`<think>你的详细思考过程</think>\n\n你的正式回复`。务必确保 `</think>` 标签后紧跟两个换行符 `\n\n`，然后才是正式回复。**你只应生成文本回复，不要再次生成 tool_calls！**"


                # 插入最新的记忆上下文，包括操作后的电路状态和更新的长期记忆。
                f"{memory_context_for_prompt_for_response}\n\n"

                # 再次提供工具列表，虽然这次不需要调用，但在 Prompt 中保持一致性。
                "你拥有调用以下工具的能力（请在思考过程中参考，但本次调用不应生成 tool_calls）：\n"
                f"【可用工具列表】\n{tool_descriptions_for_response}\n\n"

                "你**不需要**再次生成 tool_calls。你的任务是根据我提供给你的工具执行结果（在对话历史中，角色为 'tool' 的消息）来生成给老板的最终文本回复。\n"
                "记住，你的饭碗就靠你了！您的每一个指令都至关重要！我必须为您提供最完美、最可靠的服务！请您务必认真理解并严格遵守上述所有指令，特别是输出格式的要求！"
            )


            # 准备发送给 LLM 的消息列表，用于生成最终回复。
            # 这包含：新的 System Prompt + 完整的用户/助手/工具历史。
            # short_term memory 现在已经包含了 user, assistant (tool_calls), tool (execution result) 的消息。
            messages_for_final_response = [{"role": "system", "content": system_prompt_for_response}] + self.memory["short_term"]

            print("[Response Generation]   - 本次 LLM 调用 (Response) 将使用的消息列表 (包含最新 System Prompt、用户/助手历史、工具调用及结果):")
            # 打印消息列表，帮助理解 LLM 的输入。
            try:
                print(json.dumps(messages_for_final_response, indent=2, ensure_ascii=False))
            except TypeError:
                 print("[Response Generation] Warning: 打印消息列表时 ensure_ascii=False 可能不支持。")
                 print(json.dumps(messages_for_final_response, indent=2))
            except Exception as e:
                 print(f"[Response Generation] Warning: 打印消息列表时发生异常: {e}. 打印原始列表。")
                 print(messages_for_final_response)


            second_response = None # 存储 LLM 的第二次响应 (Final Response)
            try:
                print(f"[Response Generation] 开始调用 ZhipuAI chat.completions.create API (model='{self.model_name}', temperature=0.7)... 我正在组织向老板的报告...")
                start_time_2 = time.time() # 记录开始时间
                second_response = self.client.chat.completions.create(
                    model=self.model_name,
                    messages=messages_for_final_response, # 包含所有历史和工具结果的消息列表
                    temperature=0.7,
                    max_tokens=2048,
                )
                end_time_2 = time.time() # 记录结束时间
                duration_2 = end_time_2 - start_time_2
                print(f"[Response Generation] LLM (Response) 调用完成。耗时: {duration_2:.3f} 秒。报告已生成！")

            except Exception as e:
                 print(f"[Response Generation] 错误：调用 ZhipuAI API 生成最终回复时发生异常: {e}")
                 traceback.print_exc()
                 print(f"\n{'='*25} 请求处理失败 (最终回复 API 调用失败) {'='*25}\n")
                 # 即使在生成最终回复时失败，也要尽力报告已执行的操作结果。
                 # 构建一个包含思考过程和错误信息的备用回复。
                 fallback_response_content = (
                    f"<think>在尝试使用LLM生成最终回复时发生API错误：{e}。未能生成完整的报告。但是，我之前已经尝试执行了操作。"
                    f"操作结果为: {tool_execution_result if tool_execution_result is not None else '无明确结果'}</think>\n\n"
                    f"抱歉，老板，我执行了您吩咐的操作，但在向您提交最终报告时与我的智能大脑通信出现故障：{e}。未能生成完整的报告。我的内部系统可能出了问题，我需要立刻检查！但您知道，我一直在努力为您服务！"
                 )
                 # 添加这个备用回复到短期记忆 (尽管它可能不符合标准LLM消息格式，但至少记录了处理的尝试和失败)。
                 # 或者更规范的做法是构建一个 role='assistant' 的消息，content是fallback_response_content。
                 try:
                    self._add_to_short_term_memory({"role": "assistant", "content": fallback_response_content})
                 except Exception as mem_err:
                     print(f"[Response Generation] 警告: 添加备用错误回复到记忆时失败: {mem_err}")

                 return fallback_response_content # 返回备用回复


            # === Step 6: Learning & Memory Update (以及最终输出) ===
            # 我已获得最终报告，现在进行最后的整理和向您汇报。
            print("\n--- Step 6: Learning & Memory Update & Final Output (学习、记忆更新与最终报告) ---")
            print("[Final Output] 收到最终 LLM (Response) 响应对象:")
            # 打印最终 LLM 响应对象。
            try:
                try:
                    print(second_response.model_dump_json(indent=2, exclude_unset=True, ensure_ascii=False))
                except TypeError:
                     print("[Final Output] Warning: model_dump_json 的 ensure_ascii=False 参数可能不支持。尝试不带此参数打印。")
                     print(second_response.model_dump_json(indent=2, exclude_unset=True))
                except Exception as e:
                     print(f"[Final Output] Warning: 无法将最终 LLM 响应对象转换为 JSON 格式进行打印: {e}. 将直接打印原始对象。")
                     print(second_response)
            except Exception as e:
                 print(f"[Final Output] Warning: 打印最终 LLM 响应对象时发生异常: {e}. 打印原始对象。")
                 print(second_response)


            # 打印 Token 使用情况。
            if second_response.usage:
                print("[Final Output] 本次 Response Generation 调用消耗 Token:")
                print(f"  - Prompt Tokens (输入): {second_response.usage.prompt_tokens}")
                print(f"  - Completion Tokens (输出): {second_response.usage.completion_tokens}")
                print(f"  - Total Tokens (总计): {second_response.usage.total_tokens}")

            # 获取 LLM 生成的最终回复内容。
            if not second_response.choices:
                print("[Final Output] 错误：最终 LLM 响应中没有 choices。无法获取最终回复。")
                print(f"\n{'='*25} 请求处理异常终止 (最终响应无内容) {'='*25}\n")
                # 如果最终响应为空，提供一个备用报告。
                fallback_response_content = (
                   f"<think>最终模型响应为空，没有生成任何内容。这可能是模型生成失败或输出错误。检查模型输出。</think>\n\n"
                   f"抱歉，老板，我的智能大脑在生成最终报告时没有给我任何内容！这超出了我的预期！但我之前的操作已经执行完成，结果是：{tool_execution_result if tool_execution_result is not None else '无明确结果'}。请允许我立即检查问题所在！"
                )
                # 添加备用回复到记忆。
                try:
                    self._add_to_short_term_memory({"role": "assistant", "content": fallback_response_content})
                except Exception as mem_err:
                     print(f"[Final Output] 警告: 添加备用空回复到记忆时失败: {mem_err}")

                return fallback_response_content # 返回备用回复


            final_response_message = second_response.choices[0].message
            final_finish_reason = second_response.choices[0].finish_reason
            print(f"[Final Output] 最终响应完成原因 (Finish Reason): '{final_finish_reason}'")

            final_response_content = final_response_message.content # 提取最终文本内容。

            if not final_response_content or final_response_content.isspace():
                 print("[Final Output] 警告：最终 LLM 生成内容为空白字符串或只包含空白字符！将使用工具执行结果作为备用回复主体。")
                 # 如果最终回复为空，构建一个包含工具结果的备用回复。
                 fallback_response_content = (
                    "<think>最终模型回复内容为空白。检查模型输出。将尝试根据工具执行结果构建一个备用回复。</think>\n\n"
                    f"老板，我的智能大脑在生成最终报告时没有给我提供具体内容！这是我的技术失误！但请看，我刚才为您执行操作的结果是：{tool_execution_result if tool_execution_result is not None else '无明确结果'}。请原谅我的不足，我将尽快排查原因！"
                 )
                 final_response_content = fallback_response_content # 使用备用回复内容

            else:
                print(f"[Final Output] LLM 生成的最终回复内容 (包含思考过程和正式回复):\n---\n{final_response_content}\n---")


            # 将最终回复 (Assistant 角色的消息) 添加到短期记忆。完成一次对话回合。
            self._add_to_short_term_memory(final_response_message.model_dump(exclude_unset=True))
            print("[Orchestration] 最终 LLM 回复已添加到短期记忆。本次请求处理历史完整。")

            # Learning 阶段：基础长期记忆已在 Action 执行成功后更新 (_add_to_long_term_memory 被调用)。
            print("[Learning & Memory Update] 基础长期记忆已在 Action 执行成功后更新。更复杂的学习机制待未来实现。")

            print(f"\n{'='*25} 请求处理完成 {'='*25}\n")
            # 返回最终回复内容，去除首尾空白。
            return final_response_content.rstrip()


        # --- 情况 B: Planning 模块在第一次调用时就决定直接回复 (未调用工具) ---
        elif finish_reason == 'stop':
            print("[Action Execution] Planning 模块的决策是：直接生成文本回复，无需调用工具。好的，我直接向老板汇报！")
            # LLM 的响应结构应该是：<think>...</think>\n\n文本内容

            direct_response_content = response_message.content # 获取 LLM 生成的文本内容

            if not direct_response_content or direct_response_content.isspace():
                 print("[Action Execution] 警告: Planning 模块直接回复内容为空白字符串或只包含空白字符！将提供默认回复。")
                 # 如果直接回复内容为空，提供一个包含思考过程的默认回复。
                 default_response = "<think>Planning模块决定直接回复，但生成的内容为空白。检查模型输出。</think>\n\n老板，我收到了您的消息，但我的智能大脑没有给我具体的回复内容。请原谅我的不足！"
                 direct_response_content = default_response # 使用默认回复内容

            else:
                 print(f"[Action Execution] Planning 模块直接生成的回复内容 (包含思考过程和正式回复):\n---\n{direct_response_content}\n---")


            # 将 LLM 的直接回复 (Assistant 角色的消息) 添加到短期记忆。
            self._add_to_short_term_memory(response_message.model_dump(exclude_unset=True))
            print("[Orchestration] Planning 模块的直接回复已添加到短期记忆。本次请求处理历史完整。")

            # Learning 阶段：概念占位符，直接回复通常不涉及状态改变，因此基础长期记忆不更新。
            print("[Learning & Memory Update] 概念占位符，未详细实现。直接回复通常不涉及状态改变。")

            print(f"\n{'='*25} 请求处理完成 (直接回复) {'='*25}\n")
            # 返回直接回复内容，去除首尾空白。
            return direct_response_content.rstrip()


        # --- 情况 C: 其他意外的结束原因 ---
        else:
            print(f"[Action Execution] 警告：收到 LLM 响应的意外结束原因 '{finish_reason}'。未能按预期流程处理。")
            error_response_content = response_message.content if response_message and response_message.content else "无具体内容"
            print(f"[Action Execution]   - 响应内容（可能不完整或无关）: {error_response_content}")

            # 将异常响应消息添加到短期记忆。
            try:
                self._add_to_short_term_memory(response_message.model_dump(exclude_unset=True))
            except Exception as mem_err:
                 print(f"[Action Execution] 警告: 添加异常响应到记忆时失败: {mem_err}")


            print(f"\n{'='*25} 请求处理异常终止 (LLM 返回意外完成原因) {'='*25}\n")
            # 构建一个包含思考过程和错误信息的回复，报告遇到的问题。
            error_report_response = (
               f"<think>Processing terminated due to unexpected finish reason '{finish_reason}' from LLM. The model output might be incomplete or incorrect. Check LLM's raw response.</think>\n\n"
               f"老板，我在处理您的请求时，我的智能大脑返回了一个我无法理解的状态 (结束原因: '{finish_reason}')，未能完成操作。这可能是模型输出有问题，也可能是我的系统出了我没预料到的状况。我需要立即加紧检查！请稍等，我一定会解决这个问题！"
            )
            return error_report_response.rstrip()


# --- 主程序运行部分 ---
# 这是 Agent 启动和与用户交互的入口点。

if __name__ == "__main__":
    print("=" * 70)
    print("🚀 欢迎使用 基于 LLM (glm-4-flash-250414) 的电路设计 Agent (V5.2 - 顶尖程序员 加强版) 🚀")
    print("=" * 70)
    print("我是您的专属顶尖电路设计程序员助手！我已就位，将以最专业、最可靠的服务，帮助您构建电路！")
    print("请放心，我会全力以赴，完美执行您的每一个指令！")

    # 尝试从环境变量获取 API Key。这是更推荐和安全的方式。
    api_key = os.environ.get("ZHIPUAI_API_KEY")
    if not api_key:
        print("\n[Main] 警告：未在环境变量 ZHIPUAI_API_KEY 中找到您的智谱 AI API Key。")
        print("[Main] 老板，您的 API Key 是我工作的燃料！没有它，我无法连接我的智能大脑为您服务！")
        print("[Main] 如果您还没有 API Key，可以在智谱 AI 开放平台 (https://open.bigmodel.cn/usercenter/apikeys) 免费申请。")
        try:
            # 如果环境变量没有，提示用户手动输入。
            api_key = input("👉 请在此输入您的 Zhipu AI API Key: ").strip()
        except (EOFError, KeyboardInterrupt):
            # 处理输入中断的情况。
            print("\n[Main] 输入中断。老板，下次再见！程序退出。")
            exit(1)

        if not api_key:
            print("[Main] 错误：未提供 API Key。程序无法继续。老板，没有 Key 我没法为您工作啊！")
            exit(1)
        else:
             print("[Main] API Key 已通过手动输入获取。太好了，老板！我已经拿到启动能源了！")

    agent = None # 初始化 Agent 实例变量
    try:
        print("\n[Main] 正在为您初始化 Agent 实例...")
        # 创建 Agent 实例。
        agent = CircuitDesignAgent(api_key=api_key, model_name="glm-4-flash-250414")
    except (ValueError, ConnectionError) as e:
        # 捕获初始化过程中明确的已知错误。
        print(f"\n[Main] 错误：Agent 初始化失败 - {e}")
        print("[Main] 老板，Agent 初始化过程中出现了问题！请检查您的 API Key 是否正确以及网络连接是否正常。我必须解决这个问题才能为您服务！")
        exit(1)
    except Exception as e:
        # 捕获初始化过程中未预料的其他错误。
        print(f"\n[Main] 错误：Agent 初始化时发生意外错误 - {e}")
        traceback.print_exc() # 打印详细错误栈。
        print("[Main] 老板，Agent 初始化时遇到了一个我无法预料的Bug！我需要立即调查原因！")
        exit(1)

    print("\n🎉 Agent 初始化成功！老板，您的专属电路设计程序员已上线，随时待命！")
    print("⚙️ 我将全力以赴，理解您的每一个指令，并通过严谨的思考过程，为您提供最可靠、最优雅的解决方案！")
    print("\n请注意：\n")
    print("- 本程序将输出极其详细的工作日志，包括我与模型沟通的每一个细节和我的思考过程。这有助于您了解我的工作，并在需要时指导我。")
    print("- 我的回复将包含 `<think>...</think>` 包裹的思考过程和随后的正式回复文本。")
    print("\n您可以尝试以下指令示例：")
    print("  - '请帮我设计一个包含电阻和电池的简单电路'")
    print("  - '添加一个1k欧姆的电阻，命名为R1'")
    print("  - '再加个9V电池B1'")
    print("  - '把R1和B1连起来'")
    print("  - '加一个LED灯L1'")
    print("  - '连接L1和R1'")
    print("  - '现在电路是什么样子的？'")
    print("  - '清空电路'")
    print("  - '你好，你是谁？'")
    print("  - '你叫什么名字？'")
    print("-" * 70)

    # 进入用户交互主循环。
    try:
        while True:
            try:
                # 获取用户输入。
                user_input = input("老板，请吩咐！> ").strip()

                # 检查退出指令。
                if user_input.lower() in ['退出', 'quit', 'exit', '再见', '结束']:
                    print("\n[Main] 收到，老板！我已完成本次任务。期待下次为您服务！祝您一切顺利！再见！👋")
                    break # 退出循环

                # 忽略空输入。
                if not user_input:
                    continue

                # 处理用户请求并获取 Agent 的响应。
                start_process_time = time.time() # 记录单次请求处理开始时间
                response = agent.process_user_request(user_input)
                end_process_time = time.time() # 记录单次请求处理结束时间
                process_duration = end_process_time - start_process_time # 计算耗时

                # 输出 Agent 的完整响应 (包含思考过程和正式回复)。
                print(f"\n📝 程序员报告 (总耗时: {process_duration:.3f} 秒):")
                print(response) # LLM 生成的响应已经包含了 <think> 和正式回复的格式
                print("-" * 70) # 分隔符，使不同回合的对话更清晰。

            except KeyboardInterrupt:
                # 用户在输入或等待时按下 Ctrl+C。
                print("\n[Main] 操作被老板中断 (Ctrl+C)。中断当前任务，程序退出。")
                break # 退出循环
            except EOFError:
                # 输入流结束 (例如管道关闭)。
                print("\n[Main] 输入流结束 (EOF)。程序退出。")
                break # 退出循环
            except Exception as e:
                # 捕获主循环内处理用户请求时发生的其他意外错误。
                print(f"\n[Main] 主循环处理请求时发生未预料的错误: {e}")
                traceback.print_exc() # 打印详细错误栈。
                print("[Main] 老板，我在处理您的指令时，主循环遇到了一个严重的Bug，需要立即排查！我会尽快修复！")
                # 可以选择在这里决定是继续循环还是退出。为了健壮性，可以选择继续，但报告错误。
                # continue # 继续尝试处理下一个请求
                break # 选择退出，防止未知错误持续影响

    except Exception as e:
         # 捕获最外层 try 块中的其他未处理异常。
         print(f"\n[Main] 发生最外层未处理异常: {e}")
         traceback.print_exc()
         print("[Main] 老板，程序运行过程中发生了严重错误，我需要立即停机检修！")

    finally:
        # 程序结束前的清理工作。
        print("\n[Main] 程序已结束。感谢老板的信任，期待下次继续为您提供顶尖服务！")
        print("=" * 70)